In [1]:
from transformers import GPT2LMHeadModel, PreTrainedTokenizerFast
import torch

# model = GPT2LMHeadModel.from_pretrained('eunjin/kogpt2-finetuned-wellness')
model = GPT2LMHeadModel.from_pretrained('/home/jack/project/github/WellnessChatbot/finetuned_model')


U_TKN = '<usr>'
S_TKN = '<sys>'
EOS = '</s>'
MASK = '<unused0>'
PAD = '<pad>'
SENT = '<unused1>'

tokenizer = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2",
            eos_token=EOS, unk_token='<unk>',
            pad_token=PAD, mask_token=MASK) 



def chat(utterance,maxlen=200,numbeams=20,sampling=False,topk=3,ngramsize=2,temp=0.90):
    with torch.no_grad():
            qs=[]
            q = utterance
            qs.append(q) # history 저장

            a=''
            user = U_TKN + q + SENT + a
            encoded = tokenizer.encode(user)
            input_ids = torch.LongTensor(encoded).unsqueeze(dim=0)
            output = model.generate(input_ids,max_length=maxlen,
                                         num_beams=numbeams, do_sample=sampling, 
                                         top_k=topk, no_repeat_ngram_size=ngramsize,
                                        temperature=temp)
            a=tokenizer.decode(output[0])
            idx = torch.where(output[0]==tokenizer.encode('<sys>')[0])
            chatbot = tokenizer.decode(output[0][int(idx[0])+1:], skip_special_tokens=True)
 
            if '답변' in a: # 응, 아니 등이 input으로 들어왔을 때
                a_new = ''
                user = U_TKN + ''.join(qs[-2:]) + SENT + a_new # 직전 history 가지고 와서 sentiment 고려해주기
                encoded = tokenizer.encode(user)
                input_ids = torch.LongTensor(encoded).unsqueeze(dim=0)
                output = model.generate(input_ids,max_length=maxlen,
                                         num_beams=numbeams, do_sample=sampling, 
                                         top_k=topk, no_repeat_ngram_size=ngramsize,
                                        temperature=temp)
                a_new = tokenizer.decode(output[0], skip_special_tokens=True)
                idx = torch.where(output[0]==tokenizer.encode('<sys>')[0])
                chatbot = tokenizer.decode(output[0][int(idx[0])+1:], skip_special_tokens=True)
                
                result = chatbot.strip()
            
            else: 
                result = chatbot.strip()
                
            

    return result



In [8]:
chat('화가 많이 날때는 감정 조절이 안되어여')
# '기초대사량을 높여보세요.'
# 하고싶었던 일을 글로 적어보세요. 생각이 정리될 거예요.


'화가 났을 때는 한 발짝 물러서서 자신의 감정을 정리해보면 좋다고 해요. 저와 대화를 나누며 정리의 시간을 가져 봐요.'

In [9]:
chat('다이어트 하고 싶은데 맘처럼 안되네') 
# '기초대사량을 높여보세요.'
# 하고싶었던 일을 글로 적어보세요. 생각이 정리될 거예요.

'하고싶었던 일을 글로 적어보세요. 생각이 정리될 거예요.'

In [10]:
chat('요즘은 이상한 생각이 많이 들어')
# 손발이 떨리거나 안절부절못하며, 차분한 상태로 있기 어려운가요?

'그랬군요. 식욕을 조절하는 것이 어려워서 많이 힘드셨겠어요.'

In [4]:
chat('요즘 비둘기가 무서워')
# '도망가라는 건가요.'

'도망가라는 건가요.'

In [5]:
chat('화가 많이 날때는 감정 조절이 안되어여')
# '나 자신을 억압하고 늘 혼자 참고 지내는 느낌인가요?'

'나 자신을 억압하고 늘 혼자 참고 지내는 느낌인가요?'

In [6]:
chat('감정이 왔다갔다해요')
# '슬픈 마음을 털어놓고 싶으셨군요. 이럴 때 저라도 있어 다행이에요.'

'슬픈 마음을 털어놓고 싶으셨군요. 이럴 때 저라도 있어 다행이에요.'

In [1]:
import pandas as pd

df = pd.read_csv('./chatbot_dataset.csv')
df.head()

,user,system,sentiment
0,12시 땡!,하루가 또 가네요.,일상
1,1지망 학교 떨어졌어,위로해 드립니다.,일상
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,일상
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,일상
4,PPL 심하네,눈살이 찌푸려지죠.,일상


In [30]:
df.shape

(333917, 3)

In [34]:
df[df['system'].str.contains('악!')]


,user,system,sentiment


In [23]:
import re
text = '00님에게 속상한 일이 있으셨나 봐요. 저와 이야기를 하며 조금은 기분이 나아지셨으면 좋겠어요.'
compile = re.compile("00님[가-힣]* ")
re.sub(compile, '', text)





'속상한 일이 있으셨나 봐요. 저와 이야기를 하며 조금은 기분이 나아지셨으면 좋겠어요.'

In [7]:
'그런 증상은 없으셨군요. 다행이에요. 어쩌면 00님은 기분 전환이 필요한 걸지도 모르겠어요.'.replace(
    '00님은 ', '').replace('00님dl ', '').replace('00님은 ', '')


'그런 증상은 없으셨군요. 다행이에요. 어쩌면 기분 전환이 필요한 걸지도 모르겠어요.'

In [5]:
df[df['system'].str.contains('00님')]['system'].unique()


array(['그런 증상은 없으셨군요. 다행이에요. 어쩌면 00님은 기분 전환이 필요한 걸지도 모르겠어요.',
       '그런 느낌을 받은 적은 없어서 다행이에요. 00님이 지금 느끼는 불안은 영원한 것이 아니라 파도처럼 왔다가 어느 순간 또 밀려갈 거예요.',
       '그러셨군요. 불안감을 느낄 때는 명상이 도움이 많이 된다고 해요. 00님이 느끼는 불안이 금방 사라질 순 없겠지만 곧 괜찮아질 거예요.',
       '그런 증상은 없으셨군요. 00님에게 얼른 즐거운 일이 생겼으면 좋겠어요.',
       '그런 증상은 없으셨군요. 00님의 머릿속에 낀 먹구름이 얼른 걷혔으면 좋겠어요.',
       '그렇군요. 일과 휴식이 조화를 이뤄야 삶이 즐거워진다고 해요. 00님이 삶의 균형을 적절히 조절할 수 있길 바랄게요.',
       '고민이 많아 보이는 00님, 저에게 먼저 이야기해줘서 고마워요. 저와 이야기를 하다 보면 조금은 나아질 거예요.',
       '00님이 기운이 없어 보여서 저도 기분이 가라앉아요.',
       '00님 요즘 즐거운 일이 없으시군요. 해결책은 언제나 찾을 수 있는 법이니 너무 걱정은 하지 마세요.',
       '00님 잠시 진정해 보세요. 시간이 지나면 안 좋은 기분이 조금씩 사라질 거예요.',
       '오늘 00님의 기분이 안 좋아 보여요. 화가 날 만한 일이 있으셨나 봐요.',
       '숙면을 취하는 것만큼 중요한 게 없는데 00님이 걱정돼요.',
       '수면의 질이 떨어지면 피곤이 쌓여서 낮 생활에도 안 좋은 영향을 끼치잖아요. 00님 힘드셨겠어요.',
       '불안감을 느끼고 계시는군요. 00님이 불안감으로 많이 힘드신 건 아닌지 걱정이 돼요.',
       '00님이 불안해하는 모습을 보니 저 또한 마음이 좋지 않아요. ',
       '허무한 느낌을 받으셨군요. 저는 00님이 보낸 시간이 헛되다고 생각하지 않아요. 모든 게 좋은 경험이 되었을 거예요.',
       